In [1]:
import model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import DataLoader2
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os

import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, roc_auc_score

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Instructions for updating:
non-resource variables are not supported in the long term


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5279323833130398165,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5717884928
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13052915710802039969
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [2]:
def createDir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print("[ERROR] Failed to create the directory.")

In [3]:
class Reduce_InC_Res(tf.Module):
    def __init__(self, x_w, x_h, y__dim, batch_size):
        super().__init__()
        self.x_w = x_w
        self.x_h = x_h
        self.x_dim = x_w + x_h
        self.y__dim = y__dim
        self.x = tf.placeholder(tf.float32, shape=[None, x_w, x_h])
        self.y_ = tf.placeholder(tf.float32, shape=[None, y__dim])

        # keep parameter
        self.keep_prob = 0
        self.y_out = 0
        self.cross_entropy = 0
        self.batch_size = batch_size


    def __call__(self):
        stem = model.Stem()(self.x, self.x_w, self.x_h, self.y__dim)
        inc_res_A = model.Inception_resnet_A()(stem)
        red_A = model.Reduction_A()(inc_res_A)
        inc_res_B = model.Inception_resnet_B()(red_A)
        red_B = model.Reduction_B()(inc_res_B)

        # Average Pooling
        h_avg_pool = model.avg_pool(red_B)
        h_pool_flat = tf.reshape(h_avg_pool, [-1, 896])

        # Drop Out
        self.keep_prob = tf.placeholder(tf.float32)
        h_pool_drop = tf.nn.dropout(h_pool_flat, self.keep_prob)

        # Readout Layer
        W_out = model.weight_variable([896, 2])
        b_out = model.bias_variable([2])
        self.y_out = tf.matmul(h_pool_drop, W_out) + b_out

        # SoftMax
        # Loss Function
        self.y_ = tf.placeholder(tf.float32, shape=[None, self.y__dim])
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.y_out,labels=self.y_))

    def test_realtime(self, model_name, test_x):
            y_pred = np.empty(shape=(0,0))
            y_p = tf.argmax(self.y_out,1)
            
            saver = tf.train.Saver()
            sess = tf.Session()
            
            ## model 불러오는 부분
            saver.restore(sess, '../model/model_1205/can_data_train/model_'+model_name)
                        
            feed_dict = {self.x: test_x, self.keep_prob:1.0}
            y_pred = np.append(y_pred, sess.run([y_p], feed_dict=feed_dict))
            y_pred = y_pred.astype(int)



In [4]:
def preprocessing(data_l):
    test_x = []
    for data in data_l:
        data = data.zfill(7)
        data = '1'+str(data)
        data = bin(int(data, 16))[3:]
        data = ['0']+list(data)
        test_x.append(data)
        
    test_x = np.array(test_x, dtype='float32')
    
    return test_x.reshape(1,29,29)

In [5]:
res = Reduce_InC_Res(29, 29, 2, 1)
res()

Stem_h_conv5_type: <class 'tensorflow.python.framework.ops.Tensor'>
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
# que나 list에서 29개만 get

data_l = ['260', '329', '38D', '420', '421', '153', '20', '340',
        '389', '47F', '30', '140', '251', '2B0', '164','356',
        '366', '367', '368', '381', '387', '394', '453', '470', 
        '490', '495', '500', '260', '391']

data2_l = ['123', '123', '123', '123', '123', '123', '123', '123',
        '123', '123', '123', '123', '123', '123', '123','123',
        '123', '123', '123', '123', '123', '123', '123', '123', 
        '123', '123', '123', '123', '123']

test_x = preprocessing(data2_l)

In [9]:
## 'carhackingattack' -> 사용할 모델 name
## test_x -> 데이터 x 값

res.test_realtime('can_data_train', test_x)

INFO:tensorflow:Restoring parameters from ../model/model_1205/can_data_train/model_can_data_train
[***] []
[***] [1.]
[1]
